In [1]:
import re
import numpy
import pandas

In [2]:
with open('./data_metabolic_network.txt', 'r') as infile:
    data = pandas.read_csv(infile, delimiter = '\t', header = 0, comment = '#')
# data

In [3]:
for rxn in data.values:
    if 'CPLX' in rxn[0]:
        enzyme = 'cplx(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
    else:
        enzyme = 'prot(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
        
    name = rxn[1].replace('-', '_')
    substrates = rxn[2].replace('-', '_').split(', ')
    products = rxn[3].replace('-', '_').split(', ')

    LHS = []
    RHS = []
    
    for subs in substrates:
        if subs[0].isdigit():
            subs = '_' + subs
        
        if 'PER' in subs:
            LHS.append('met(name = \'{:s}\', loc = \'per\')'.format(subs.replace('PER_', '')))
        else:
            LHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(subs))

    for prod in products:
        if prod[0].isdigit():
            prod = '_' + prod
        
        if 'PER' in prod:
            RHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(prod))
        else:
            RHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(prod))
            
    LHS = ' +\n    '.join(LHS)
    RHS = ' +\n    '.join(RHS)
        
    if rxn[0] == 'spontaneous':
        Rule = 'Rule(\'{:s}\',\n'\
               '    {:s} |\n'\
               '    {:s},\n' \
               '    Parameter(\'fwd_{:s}\', 1),\n' \
               '    Parameter(\'rvs_{:s}\', 1))' \
               .format(name, LHS, RHS, name, name)
    
    else: # need an enzyme
        Rule = 'Rule(\'{:s}\',\n'\
               '    {:s} +\n'\
               '    {:s} |\n'\
               '    {:s} +\n'\
               '    {:s},\n' \
               '    Parameter(\'fwd_{:s}\', 1),\n' \
               '    Parameter(\'rvs_{:s}\', 0))' \
               .format(name, enzyme, LHS, enzyme, RHS, name, name)
        
    print(Rule)
    print()

Rule('LACTOSE_MUTAROTATION',
    met(name = 'alpha_lactose', loc = 'cyt') |
    met(name = 'beta_lactose', loc = 'cyt'),
    Parameter('fwd_LACTOSE_MUTAROTATION', 1),
    Parameter('rvs_LACTOSE_MUTAROTATION', 1))

Rule('GALACTOSE_MUTAROTATION',
    met(name = 'alpha_GALACTOSE', loc = 'cyt') |
    met(name = 'beta_GALACTOSE', loc = 'cyt'),
    Parameter('fwd_GALACTOSE_MUTAROTATION', 1),
    Parameter('rvs_GALACTOSE_MUTAROTATION', 1))

Rule('GLUCOSE_MUTAROTATION',
    met(name = 'alpha_glucose', loc = 'cyt') |
    met(name = 'beta_glucose', loc = 'cyt'),
    Parameter('fwd_GLUCOSE_MUTAROTATION', 1),
    Parameter('rvs_GLUCOSE_MUTAROTATION', 1))

Rule('TRANS_RXN_24',
    prot(name = 'lacY') +
    met(name = 'PROTON', loc = 'per') +
    met(name = 'alpha_lactose', loc = 'per') |
    prot(name = 'lacY') +
    met(name = 'PROTON', loc = 'cyt') +
    met(name = 'alpha_lactose', loc = 'cyt'),
    Parameter('fwd_TRANS_RXN_24', 1),
    Parameter('rvs_TRANS_RXN_24', 0))

Rule('TRANS_RXN_24_beta',